# Final Project: Green Transit Solutions and Route Planner for Metro Commuters

Laure | 


## Description

The Green Transit Solutions and Route Planner for Metro Commuters aims to visualize routes for transit planning. The project initializes the map and loads various layers such as transit lines (green and blue line), stops, and markers using ipyleaflet and geopandas. The map interactivity is facilitated through map clicks to add markers and a button to calculate the routes using Google Maps API for directions. Beyond the visualizations, users can choose between driving and transit mode, input origin and destination either via addresses or markers, select time preferences, preference for routes, and see options under the layers legend. This project dynamically updates the map to visualize selected routes, removes old layers, and adds new polyline layers to represent the new route. The project made this interface user friendly with widgets to enhance the overall user experience. 

## Problem Statement

This code significantly enhances the user experience compared to traditional metro trip planners. By integrating interactive map markers, real-time traffic updates (including pessimistic, optimistic, and best-guess scenarios), and eco-friendly options that prioritize fewer connections and more walking to transit stations, this project revolutionizes transit planning.One of the key advantages for users is the dynamic updating feature, which ensures that they always have the latest information about their journey. This eliminates the frustration of outdated or inaccurate data often encountered with conventional trip planners. Additionally, the inclusion of the Google Maps API not only provides accurate directions but also offers insights into real-time events that may impact travel plans. By simplifying the planning process and offering a more comprehensive set of features, this code empowers users to make informed decisions about their transit routes. Whether it's choosing the most efficient path, staying updated on traffic conditions, or opting for eco-friendly travel options, this project aims to enhance the overall transit experience for users.

# Installations

In [15]:
!pip install ipywidgets --upgrade

## Import Statements

In [16]:
from ipyleaflet import Map, GeoData, basemaps, LayersControl, Marker, Polyline
from shapely.geometry import LineString
import geopandas as gpd
import pandas as pd
from ipywidgets import HTML, Dropdown, VBox, RadioButtons, Text, Button, DatetimePicker
import requests
from datetime import timezone
import pytz

## Merging Shapefile and CSV Data for Transit Stops

In [17]:
#file paths
shapefile_path = "Stops/TransitStops.shp"
#opened STOP_RouteListByStop.dbf in arcgis and saved it as csv
csv_file_path = "Stops/routelist.csv"
#load the shapefile
gdf = gpd.read_file(shapefile_path)
#load the CSV file
df = pd.read_csv(csv_file_path)
#merge the GeoDataFrame with the DataFrame based on 'site_id'
merged_gdf = gdf.merge(df, on='site_id')
#each row in 'merged_gdf' corresponds to a point for each row in the CSV, and including the associated route information
#save the merged GeoDataFrame to a new shapefile
output_shapefile_path = "Stops/AllStops.shp"
merged_gdf.to_file(output_shapefile_path)
#paths to green and blue line shapefiles
green_line_shapefile = 'Lines/green_line.shp'
blue_line_shapefile = 'Lines/blue_line.shp'
#load the shapefiles using GeoPandas (makes easy to display)
green_gdf = gpd.read_file(green_line_shapefile)
blue_gdf = gpd.read_file(blue_line_shapefile)
#reproject the GeoDataFrames to WGS84 (EPSG:4326)
green_gdf = green_gdf.to_crs(epsg=4326)
blue_gdf = blue_gdf.to_crs(epsg=4326)
#create GeoData objects from the GeoDataFrames (layers)
green_geo_data = GeoData(
    geo_dataframe=green_gdf,
    style={'color': 'green', 'opacity': 1, 'weight': 2},
    name='Green Line'
)
#create GeoData objects from the GeoDataFrames (layers)
blue_geo_data = GeoData(
    geo_dataframe=blue_gdf,
    style={'color': 'blue', 'opacity': 1, 'weight': 2},
    name='Blue Line'
)
#load the shapefile using geopandas for all stops shapefile
gdf = gpd.read_file("Stops/AllStops.shp")
#reproject the GeoDataFrame to WGS84 (EPSG:4326)
gdf = gdf.to_crs(epsg=4326)
#create a GeoData object from the GeoDataFrame for all stops shapefile
geo_data = GeoData(
    geo_dataframe=gdf,
    style={'color': 'pink', 'opacity': 1, 'weight': 1.9, 'fillColor': 'pink', 'fillOpacity': 0.6},
    point_style={'radius': 2, 'fillColor': 'red', 'fillOpacity': 0.6, 'color': 'pink', 'weight': 1},
    name='Stops'
)

## Global Settings

In [18]:
#initialize an empty list to store the markers
markers = []
#initialize HTML widget to display directions
directions_widget = HTML()

## Google API Call

Google Documentation: https://developers.google.com/maps/documentation/directions/get-directions

In [19]:
#this fucntion decodes googles polyline shape (reads in text and returns a shape)
def decode_polyline(polyline_str):
    index, lat, lng = 0, 0, 0
    coordinates = []
    changes = {'latitude': 0, 'longitude': 0}
    while index < len(polyline_str):
        for unit in ['latitude', 'longitude']:
            shift, result = 0, 0
            while True:
                byte = ord(polyline_str[index]) - 63
                index += 1
                result |= (byte & 0x1f) << shift
                shift += 5
                if not byte >= 0x20:
                    break
            if result & 1:
                changes[unit] = ~(result >> 1)
            else:
                changes[unit] = (result >> 1)
        lat += changes['latitude']
        lng += changes['longitude']
        coordinates.append((lat / 1E5, lng / 1E5))
    return coordinates

#this function calls google api, then makes directions (text list of directions), then creates line
def get_readable_directions(origin, destination, mode, api_key, departure_time=None, arrival_time = None, traffic_model = None, routing_preference = None):
    #paramters is settings for api call
    parameters = f"origin={origin}&destination={destination}&mode={mode}&key={api_key}"
    #these if and else statements check if user inputed time or traffic models and adds those to parameters
    if departure_time:
        parameters += f"&departure_time={departure_time}"
    elif arrival_time:
        parameters += f"&arrival_time={arrival_time}"
    if mode == 'driving' and traffic_model and departure_time:
        parameters += f"&traffic_model={traffic_model}"
    if mode == 'transit' and routing_preference:
        parameters += f"&transit_routing_preference={routing_preference}"
    #makes directions api url
    directions_url = f"https://maps.googleapis.com/maps/api/directions/json?{parameters}"
    #gets api call
    response = requests.get(directions_url)
    #makes into json (dictionary)
    directions = response.json()
    #checks if api call is good
    if directions["status"] == "OK":
        #index at very first route (giving first route to travel) (first direction)
        route = directions["routes"][0]
        #all the points along the line
        polyline = route["overview_polyline"]["points"]
        #decodes polyline into a readable format
        decoded_polyline = decode_polyline(polyline)
        #turns points into line (dictionary is points)
        line = LineString(decoded_polyline)
        #creates layer to display polyline
        new_polyline_layer = Polyline(locations=[list(coord) for coord in line.coords], color="blue", weight=5, fill = False, name = 'route')
        #remove old polyline layers if exist
        for layer in m.layers[:]:
            if isinstance(layer, Polyline):
                m.remove_layer(layer)
        #adds line to map
        m.add_layer(new_polyline_layer)
        #creating a list of directions
        directions_text = ""
        #legs = entire trip
        for leg in route["legs"]:
            directions_text += f"<b>Total distance:</b> {leg['distance']['text']}<br>"
            #if we are driving uses traffic model and departue time
            if mode == 'driving' and traffic_model and departure_time:
                directions_text += f"<b>Estimated duration:</b> {leg['duration_in_traffic']['text']}<br>"
            #otherwise uses just regular duration (tranist) and not duration_in_traffic (driving)
            else:
                directions_text += f"<b>Estimated duration:</b> {leg['duration']['text']}<br>"
            directions_text += "<b>Directions:</b><br>"
            #each segment of traveling
            for step in leg["steps"]:
                #for transit gets the transit line name and stops
                if step.get("travel_mode") == "TRANSIT":
                    details = step["transit_details"]
                    line = details["line"]["short_name"] if "short_name" in details["line"] else details["line"]["name"]
                    directions_text += f"- Take {line} from {details['departure_stop']['name']} to {details['arrival_stop']['name']}<br>"
                #for driving (more structured directions)
                else:
                    directions_text += f"- {step['html_instructions']}<br>"
            directions_text += "<br>"
        directions_widget.value = directions_text
    else:
        directions_widget.value = "Could not retrieve directions. Please check your input or try again later."

## Interactions

In [ ]:
#checking when the user clicks and when the user clicks markers are added
def handle_map_interaction(**kwargs):
    if kwargs.get('type') == 'click':
        coords = kwargs.get('coordinates')
        if len(markers) >= 2:
            m.remove_layer(markers.pop(0))
        marker = Marker(location=coords, name = 'marker')
        markers.append(marker)
        m.add_layer(marker)
        
#calculate route button is clicked-- checks if we are driving or doing transit-- and add different optional parameter into api call
def handle_button_click(button):
    #mode is form of transit
    mode = modes_radiobutton.value
    #optional parameters
    time_option = None
    departure_time = None
    arrival_time = None
    traffic_model = None
    routing_preference = None
    #checks on optional parameters (assigns traffic model if looking at driving)
    if mode == 'driving':
        traffic_model = traffic_model_dropdown.value
    #routing preference
    if mode == 'transit':
        routing_preference = transit_routing_preference_dropdown.value
    #time picker to tranist
    if mode == 'transit' and transit_time_picker.value:
        user_time = transit_time_picker.value
        time_option = transit_time_option_radiobutton.value
        #convert the localized time to UTC
        utc_time = user_time.astimezone(pytz.utc)
        time_value = int(utc_time.timestamp())
        #depart/arrival for tranist-- checking button that was clicked
        if time_option == 'Depart at':
            departure_time = time_value
        else:
            arrival_time = time_value
    #time paramters for driving
    elif mode == 'driving' and departure_time_picker.value:
        user_time = departure_time_picker.value
        # Convert the localized time to UTC
        utc_time = user_time.astimezone(pytz.utc)
        departure_time = int(utc_time.timestamp())
    #button    
    route_by = route_selection_radiobutton.value
    #api key (do not share)
    api_key = ''
    
    #enter address
    if route_by == 'Enter Addresses':
        #.strip removes spaces
        origin_address = origin_text.value.strip()
        destination_address = destination_text.value.strip()
        #checks if both addresses are present (two correct addresses in address boxs) and calls api to perform transit operations 
        if origin_address and destination_address:
            get_readable_directions(origin_address, destination_address, mode, api_key, departure_time, arrival_time, traffic_model, routing_preference)
        else:
            directions_widget.value = "Please enter both origin and destination addresses."
    #checks if markers are used instead of addresses
    #ensures 2 markers are present
    elif route_by == 'Use Markers' and len(markers) == 2:
        origin_coords = markers[0].location
        destination_coords = markers[1].location
        origin = f"{origin_coords[0]},{origin_coords[1]}"
        destination = f"{destination_coords[0]},{destination_coords[1]}"
        #api call with coordinates instead of addresses
        get_readable_directions(origin, destination, mode, api_key, departure_time, arrival_time, traffic_model, routing_preference)
    else:
        directions_widget.value = "Please select both origin and destination points on the map."
#create a map with an appropriate basemap
center = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())
m = Map(center=center, zoom=10, basemap=basemaps.OpenStreetMap.Mapnik)

/tmp/ipykernel_229/1149717714.py:72: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())
/tmp/ipykernel_229/1149717714.py:72: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())


## Widgets

In [21]:
#handling widgets
#add the GeoData layers to the map
m.add_layer(green_geo_data)
m.add_layer(blue_geo_data)
#add the GeoData layer to the map
m.add_layer(geo_data)
#add layers control for toggling layers
m.add_control(LayersControl())
#handle map interaction for clicks
m.on_interaction(handle_map_interaction)
#create a RadioButton for mode selection
modes_radiobutton = RadioButtons(
    options=['driving', 'transit'],
    description='Mode:',
    disabled=False
)
#button use markers or enter addresses
route_selection_radiobutton = RadioButtons(
    options=['Use Markers', 'Enter Addresses'],
    description='Route by:',
    disabled=False
)
#initialize the text widgets for address input
origin_text = Text(
    value='',
    placeholder='ADDRESS AND CITY',
    description='Origin:',
    disabled=False
)
#initialize the text widgets for address input
destination_text = Text(
    value='',
    placeholder='ADDRESS AND CITY',
    description='Destination:',
    disabled=False
)
#initialize the DatetimePicker widget for departure time selection
departure_time_picker = DatetimePicker(
    description='Departure Time:',
    disabled=False
)
#initialize the Dropdown widget for traffic model selection
traffic_model_dropdown = Dropdown(
    options=['best_guess', 'optimistic', 'pessimistic'],
    value='best_guess',  # Default value
    description='Traffic:',
    disabled=False
)
#initialize the DatetimePicker widget for departure or arrival time selection for transit
transit_time_picker = DatetimePicker(
    description='Time:',
    disabled=False
)
#create a RadioButton for selecting "Depart at" or "Arrive by" for transit
transit_time_option_radiobutton = RadioButtons(
    options=['Depart at', 'Arrive by'],
    description='Transit Time:',
    disabled=False
)
#initialize the Dropdown widget for routing preference selection
transit_routing_preference_dropdown = Dropdown(
    options=[('Fewer Transfers', 'fewer_transfers'), ('Less Walking', 'less_walking')],
    value='fewer_transfers',  # Default value
    description='Preference:',
    disabled=False
)
#function to show/hide the departure time picker based on the selected mode
def toggle_departure_time_picker(change):
    if change['new'] == 'driving':
        departure_time_picker.layout.display = 'flex'
    else:
        departure_time_picker.layout.display = 'none'
        traffic_model_dropdown.layout.display = 'none'
    if change['new'] == 'transit':
        transit_time_picker.layout.display = 'flex'
        transit_time_option_radiobutton.layout.display = 'flex'
        transit_routing_preference_dropdown.layout.display = 'flex'
    else:
        transit_time_picker.layout.display = 'none'
        transit_time_option_radiobutton.layout.display = 'none'
        transit_routing_preference_dropdown.layout.display = 'none'

#function to toggle the visibility of the traffic model selection box based on the departure time picker's value
def toggle_traffic_model_visibility(change):
    if modes_radiobutton.value == 'driving' and departure_time_picker.value is not None:
        traffic_model_dropdown.layout.display = 'flex'
    else:
        traffic_model_dropdown.layout.display = 'none'
#hides ones that need to be hidden at the start for better user experience
transit_time_picker.layout.display = 'none'
transit_time_option_radiobutton.layout.display = 'none'
transit_routing_preference_dropdown.layout.display = 'none'
traffic_model_dropdown.layout.display = 'none'
#looks for changes in the mode selection to toggle the departure time picker visibility
modes_radiobutton.observe(toggle_departure_time_picker, names='value')
#looks for changes in the mode selection to toggle the traffic model picker visibility
departure_time_picker.observe(toggle_traffic_model_visibility, names='value')
#create a Button for initiating route calculation
calculate_route_button = Button(description="Calculate Route")
#assign the button click event to the route calculation function
calculate_route_button.on_click(handle_button_click)
#create and display the title
title_widget = HTML(value='<h2 style="text-align:center;">Interactive Transit Map</h2>')
#update the widget box to include the new widgets for transit time selection
widget_box = VBox([title_widget, m, route_selection_radiobutton, origin_text, destination_text, modes_radiobutton, departure_time_picker, traffic_model_dropdown, transit_time_picker, transit_time_option_radiobutton, transit_routing_preference_dropdown, calculate_route_button, directions_widget])
#display the VBox
display(widget_box)

## Challenges

This project encountered several challenges during development. One significant challenge was implementing our custom routing system. The dots representing transit stops were not aligning perfectly with the transit lines, leading to imperfect overlays and segmented lines between dots instead of a continuous line and lead to the Google API being used. Another challenge was the incorrect assumption of the system regarding the user's time zone. The project assumed UTC time, instead of accounting for the user's local time zone, such as Central time. Finally, during the debugging portion of this project, time estimates were not accurate and were corrected.

## Solutions

Additionally, in the future, this project could benefit from incorporating data about major events like concerts and sports games in cities, leading to more accurate predictive models. Transitioning the project into a web application with a dashboard interface would enhance user-friendliness and accessibility. Furthermore, expanding the project to include bus routes would provide users with a comprehensive view of transit options. Ultimately, these enhancements would empower users to plan their routes more effectively, making their transit experience smoother and more convenient. Despite these potential future improvements, it's important to note that the current project still represents a major step forward in transit planning. Its integration of interactive map markers, real-time traffic information, and eco-friendly options already provides users with valuable tools to plan their journeys efficiently.